<a href="https://colab.research.google.com/github/joeygoon/humandetection/blob/master/human_detection_in_google_colab_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Object Detection Demo


Welcome to the object detection inference walkthrough! This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the installation instructions before you start.

## Installing necessary packages to run Tensorflow-Object Detetion_API in colab

* install protobuf and cython
* clone tensoflow models github
* build protobuf
* at the end test installation, so you must see pass 13 test in a short time

In [0]:
!git clone https://github.com/Amin-Tgz/Tensorflow-Object-Detection-API-google-colab

Cloning into 'Tensorflow-Object-Detection-API-google-colab'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 13 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
#!pip install jupyter
#!pip install matplotlib

!git clone https://github.com/tensorflow/models.git

%cd /content/models/research

!protoc object_detection/protos/*.proto --python_out=.

%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

!python object_detection/builders/model_builder_test.py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-pil is already the newest version (5.1.0-1).
python-tk is already the newest version (2.7.15~rc1-1).
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-lxml is already the newest version (4.2.1-1ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
fatal: destination path 'models' already exists and is not an empty directory.
/content/models/research
env: PYTHONPATH=/content/models/research:/content/models/research/slim

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

............s...
----------------------------------------


## Imports


* remove check tf version , by default last version installed on colab



In [0]:
from google.colab.patches import cv2_imshow 
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile


print('tensorflow version = ',tf.__version__)

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops


tensorflow version =  1.13.1


* change directory to use utils 

In [0]:
!pwd

/content/models/research


In [0]:
%cd ./object_detection/

from utils import label_map_util

from utils import visualization_utils as vis_util

/content/models/research/object_detection


In [0]:
# This is needed to display the images.
%matplotlib inline

## Model preparation

In [0]:
!unzip /content/models/research/object_detection/images.zip
!unzip /content/models/research/object_detection/training.zip
! unzip /content/models/research/object_detection/doc.zip
! unzip /content/models/research/object_detection/inference_graph.zip
! unzip /content/models/research/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.zip




Archive:  /content/models/research/object_detection/images.zip
   creating: images/test/
  inflating: images/test/000001.jpg  
  inflating: images/test/000001.xml  
  inflating: images/test/000084.jpg  
  inflating: images/test/000084.xml  
  inflating: images/test/000179.jpg  
  inflating: images/test/000179.xml  
  inflating: images/test/000300.jpg  
  inflating: images/test/000300.xml  
  inflating: images/test/000444.jpg  
  inflating: images/test/000444.xml  
  inflating: images/test/000540.jpg  
  inflating: images/test/000540.xml  
  inflating: images/test/000660.jpg  
  inflating: images/test/000660.xml  
  inflating: images/test/000900.jpg  
  inflating: images/test/000900.xml  
  inflating: images/test/001116.jpg  
  inflating: images/test/001116.xml  
  inflating: images/test/001356.jpg  
  inflating: images/test/001356.xml  
  inflating: images/test/001428.jpg  
  inflating: images/test/001428.xml  
  inflating: images/test/001608.jpg  
  inflating: images/test/001608.xml  

In [0]:
!python xml_to_csv.py
!python generate_tfrecord.py --csv_input=/content/models/research/object_detection/images/train_labels.csv --image_dir=/content/models/research/object_detection/images/train --output_path=/content/models/research/object_detection/train.record
!python generate_tfrecord.py --csv_input=/content/models/research/object_detection/images/test_labels.csv --image_dir=/content/models/research/object_detection/images/test --output_path=/content/models/research/object_detection/test.record


Successfully converted xml to csv.
Successfully converted xml to csv.
Successfully created the TFRecords: /content/models/research/object_detection/train.record
Successfully created the TFRecords: /content/models/research/object_detection/test.record


In [0]:
!pwd


/content/models/research/object_detection


In [0]:
!python train.py --logtostderr --train_dir=/content/models/research/object_detection/training/ --pipeline_config_path=/content/models/research/object_detection/training/faster_rcnn_inception_v2_pets.config

python3: can't open file 'train.py': [Errno 2] No such file or directory


In [0]:
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path /content/models/research/object_detection/training/faster_rcnn_inception_v2_pets.config --trained_checkpoint_prefix /content/models/research/object_detection/training/model.ckpt-31537 --output_directory /content/models/research/object_detection/inference_graph


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
Instructions for updating:
Use `tf.profiler.profile(graph, run_meta, op_log, cmd, options)`. Build `options` with `tf.profiler.ProfileOptionBuilder`. See README.md for details
Instructions for updating:
Use tf.compat.v1.graph_util.remove_training_nodes
247 ops no flops stats due to incomplete shapes.
Parsing Inputs...
Incomplete shape.

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops   

In [0]:
!zip -r /content/file.zip /content/models/research/object_detection

  adding: content/models/research/object_detection/ (stored 0%)
  adding: content/models/research/object_detection/export_tflite_ssd_graph_lib_test.py (deflated 84%)
  adding: content/models/research/object_detection/predictors/ (stored 0%)
  adding: content/models/research/object_detection/predictors/convolutional_keras_box_predictor.py (deflated 66%)
  adding: content/models/research/object_detection/predictors/convolutional_box_predictor.py (deflated 76%)
  adding: content/models/research/object_detection/predictors/__pycache__/ (stored 0%)
  adding: content/models/research/object_detection/predictors/__pycache__/mask_rcnn_box_predictor.cpython-36.pyc (deflated 55%)
  adding: content/models/research/object_detection/predictors/__pycache__/convolutional_keras_box_predictor.cpython-36.pyc (deflated 53%)
  adding: content/models/research/object_detection/predictors/__pycache__/rfcn_box_predictor.cpython-36.pyc (deflated 53%)
  adding: content/models/research/object_detection/predictors

In [0]:
!zip -r /content/models.zip /content/models

  adding: content/models/ (stored 0%)
  adding: content/models/.gitmodules (deflated 24%)
  adding: content/models/tutorials/ (stored 0%)
  adding: content/models/tutorials/embedding/ (stored 0%)
  adding: content/models/tutorials/embedding/word2vec_test.py (deflated 51%)
  adding: content/models/tutorials/embedding/word2vec_optimized.py (deflated 66%)
  adding: content/models/tutorials/embedding/word2vec_optimized_test.py (deflated 51%)
  adding: content/models/tutorials/embedding/word2vec_kernels.cc (deflated 70%)
  adding: content/models/tutorials/embedding/word2vec_ops.cc (deflated 58%)
  adding: content/models/tutorials/embedding/word2vec.py (deflated 67%)
  adding: content/models/tutorials/embedding/__init__.py (deflated 45%)
  adding: content/models/tutorials/embedding/README.md (deflated 52%)
  adding: content/models/tutorials/rnn/ (stored 0%)
  adding: content/models/tutorials/rnn/ptb/ (stored 0%)
  adding: content/models/tutorials/rnn/ptb/ptb_word_lm.py (deflated 69%)
  addin

In [0]:
!zip -r /content/inference_graph.zip /content/models/research/object_detection/inference_graph

  adding: content/models/research/object_detection/inference_graph/ (stored 0%)
  adding: content/models/research/object_detection/inference_graph/checkpoint (deflated 42%)
  adding: content/models/research/object_detection/inference_graph/pipeline.config (deflated 68%)
  adding: content/models/research/object_detection/inference_graph/model.ckpt.meta (deflated 94%)
  adding: content/models/research/object_detection/inference_graph/saved_model/ (stored 0%)
  adding: content/models/research/object_detection/inference_graph/saved_model/variables/ (stored 0%)
  adding: content/models/research/object_detection/inference_graph/saved_model/saved_model.pb (deflated 8%)
  adding: content/models/research/object_detection/inference_graph/.ipynb_checkpoints/ (stored 0%)
  adding: content/models/research/object_detection/inference_graph/frozen_inference_graph.pb (deflated 8%)
  adding: content/models/research/object_detection/inference_graph/model.ckpt.data-00000-of-00001 (deflated 7%)
  adding: c

In [0]:
! unzip /content/models/research/object_detection/training.zip

Archive:  /content/models/research/object_detection/training.zip
  inflating: training/checkpoint     
  inflating: training/events.out.tfevents.1554571905.5293cabc849e  
  inflating: training/events.out.tfevents.1554581825.5293cabc849e  
  inflating: training/faster_rcnn_inception_v2_pets.config  
  inflating: training/graph.pbtxt    
  inflating: training/labelmap.pbtxt  
  inflating: training/model.ckpt-24824.data-00000-of-00001  
  inflating: training/model.ckpt-24824.index  
  inflating: training/model.ckpt-24824.meta  
  inflating: training/model.ckpt-26501.data-00000-of-00001  
  inflating: training/model.ckpt-26501.index  
  inflating: training/model.ckpt-26501.meta  
  inflating: training/model.ckpt-28179.data-00000-of-00001  
  inflating: training/model.ckpt-28179.index  
  inflating: training/model.ckpt-28179.meta  
  inflating: training/model.ckpt-29858.data-00000-of-00001  
  inflating: training/model.ckpt-29858.index  
  inflating: training/model.ckpt-29858.meta  
  infla

In [0]:
! python Object_detection_image.py

2019-04-08 18:14:16.978361: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-04-08 18:14:16.978705: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x2ef5080 executing computations on platform Host. Devices:
2019-04-08 18:14:16.978748: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-04-08 18:14:17.079020: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-04-08 18:14:17.079639: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x2ef5a20 executing computations on platform CUDA. Devices:
2019-04-08 18:14:17.079682: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla K80, Compute Capability 3.7
2019-04-08 18:14:17.080061: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found d

In [0]:
from google.colab.patches import cv2_imshow

In [0]:

# Import packages
import os
import cv2
import numpy as np
import tensorflow as tf
import sys


# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

# Import utilites
from utils import label_map_util
from utils import visualization_utils as vis_util

# Name of the directory containing the object detection module we're using
MODEL_NAME = 'inference_graph'
IMAGE_NAME = 'human.jpg'

# Grab path to current working directory
CWD_PATH = os.getcwd()

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,'training','labelmap.pbtxt')

# Path to image
PATH_TO_IMAGE = os.path.join(CWD_PATH,IMAGE_NAME)

# Number of classes the object detector can identify
NUM_CLASSES = 1

# Load the label map.
# Label maps map indices to category names, so that when our convolution
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

# Load image using OpenCV and
# expand image dimensions to have shape: [1, None, None, 3]
# i.e. a single-column array, where each item in the column has the pixel RGB value
image = cv2.imread(PATH_TO_IMAGE)
image_expanded = np.expand_dims(image, axis=0)

# Perform the actual detection by running the model with the image as input
(boxes, scores, classes, num) = sess.run(
    [detection_boxes, detection_scores, detection_classes, num_detections],
    feed_dict={image_tensor: image_expanded})

# Draw the results of the detection (aka 'visulaize the results')

vis_util.visualize_boxes_and_labels_on_image_array(
    image,
    np.squeeze(boxes),
    np.squeeze(classes).astype(np.int32),
    np.squeeze(scores),
    category_index,
    use_normalized_coordinates=True,
    line_thickness=8,
    min_score_thresh=0.80)

# All the results have been drawn on image. Now display the image.
cv2.imshow('Object detector', image)

# Press any key to close the image
cv2.waitKey(0)

# Clean up
cv2.destroyAllWindows()


DisabledFunctionError: ignored